## Agile Hardware Design
***
# FIRRTL

## Prof. Scott Beamer
### sbeamer@ucsc.edu

## [CSE 293](https://classes.soe.ucsc.edu/cse293/Winter22/)

## Plan for Today

* Motivation
* FIRRTL Overview
* Example Design in FIRRTL
* Example Optimizations / Transforms
* Future of Exposed IRs in Hardware Design

## Loading The Chisel Library Into a Notebook

In [ ]:
val path = System.getProperty("user.dir") + "/../resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

## What Have We Been Doing with Hardware Generators?

* Concretely: use _programming_ (in Scala) to orchestrate _hardware instantiation_ (in Chisel)
  * Can think of all of the Chisel commands we have learned as instantiating hardware (including connections)
* Hardware generators allow us to _automate_ construction of hardware
  * Part of the design is generated when it is needed rather than at the time it is written
* Developers need to imagine possible situations and provide parameters (with generator support) to implement them
  * Generation capabilities and implementation is specific to that generator

## What About Reusing an Optimization?

* Imagine for your generator, you make an optimization, and want to generalize it to use in other generators
* You could to encapsulate the optimization in clever Scala
  * Could use spiffy features like generic types and/or inheritance
  * Everyone that uses the optimization needs to modify code significantly to instantiate that optimization
  * As the optimization implementor, will need to figure out a sufficiently flexible pattern to support everyone
  * Compatability/composability with similar optimizations will be a challenge
* **Q:** _Why is this hard?_
  * **A:** input/interface to optimization is too broad and undefined (arbitrary Scala)

## Is There An Alternate Way to Package an Optimization?

* Prior approach (templated Scala) applies optimization while constructing the design
* _Alternative:_ construct design first, then optimize existing design
  * Greatly simplifies input/interface - now a HW design instead of arbitrary Scala
  * Compatability/composability with similar optimizations easier because their input & output formats are now the same
* _Summary:_ some optimizations/transformations are best implemented in tools rather than manually in design or generator
  * More reusable, and may be able to act on more information
  * Blurs line between putting automation/smarts in generator or tools

## Custom Transformations Ease Complexity-Difficulty Tradeoff

* Static design - is easy to get started, but complexity rapidly increases
* Generator design increases scope of practical transformations
* Generator + custom transformations reduces challenge at top end

<img src="images/comp-diff.svg" alt="complexity difficulty tradeoffs" style="width:50%;margin-left:auto;margin-right:auto"/>

## Introducing Hardware Intermediate Representations

* _Intermediate representation (IR)_ expresses a design in a well-defined syntax
  * Typically think of it as a graph with varying node types
  * Using IRs is an established best practice from compliers/PL
* IRs make it easy for others to make/modify/reuse tools
  * _Want an optimization?_ -> add a new _pass_ to the middle of the tool flow
  * _Want a new language?_ -> make a new _frontend_ and reuse the rest
  * _Want a new target?_ -> make a new _backend_ and reuse the rest
* Historically, hardware tools kept IRs internal, and did not even formalize/specify them
  * Specifying and externalizing IR is a key enabler

## Introducing Flexible Intermediate Representation for RTL (FIRRTL)

* IR for Chisel, but used by other projects too
* FIRRTL greatly improved development/reliability of Chisel
  * chisel2 (prior version) was monolithic and complex
    * complexity introduced bugs and discouraged contributions
  * Rewrite with FIRRTL (and IR using passes) much easier to work with
* "FIRRTL" can be ambiguous, because refers to
  * FIRRTL specification/format
  * a design in FIRRTL (.fir as a file)
  * `firrtl` library (which processes FIRRTL)

<img src="https://raw.githubusercontent.com/chipsalliance/firrtl/master/doc/images/firrtl_logo.svg" alt="firrtl logo" style="width:40%;margin-left:auto;margin-right:auto"/>

## Chisel Tool Flow

<img src="images/flow.svg" alt="complexity difficulty tradeoffs" style="width:95%;margin-left:auto;margin-right:auto"/>

## The IR Within FIRRTL

* Composition of FIRRTL components:
  * A hardware design is a _circuit_ composed of _modules_
  * A _module_ contains _ports_ and _statements_
  * A _statement_ can contain _expressions_ or other _statements_
  * An _expression_ can contain _expressions_

<img src="images/types.svg" alt="FIRRTL node types" style="width:40%;margin-left:auto;margin-right:auto"/>

## IR Node Types in FIRRTL


* IR nodes extend abstract classes in FIRRTL
  * _Circuit_ - top level of design
  * _Module_ - a component
  * _Port_ - an input or output for a module
  * _Type_ - e.g. `UInt`, `SInt`, `Clock`
  * _Statement_ - connection, declaration, or instantiation (e.g. DefWire, Connect...)
  * _Expression_ - reference, literal, or operation (e.g. Ref, Mux, DoPrim...)

## FIRRTL Example - Simple Delay Module (1/3)

In [ ]:
class Delay extends Module {
    val io = IO(new Bundle {
        val in = Input(UInt(4.W))
        val out = Output(UInt(4.W))
    })
    io.out := RegNext(io.in)
}

// println(getVerilog(new Delay))
println(getFirrtl(new Delay))

## FIRRTL Example - Simple Delay Module (2/3)

In [ ]:
import firrtl.Parser
val firrtlSerialization = getFirrtl(new Delay)
val firrtlAST = firrtl.Parser.parse(firrtlSerialization.split("\n").toIterator, Parser.GenInfo("file.fir"))

// println(firrtlAST)
println(stringifyAST(firrtlAST))

## FIRRTL Example - Simple Delay Module (3/3)

<img src="images/delay.svg" alt="Delay FIRRTL example" style="width:90%;margin-left:auto;margin-right:auto"/>

["Reusability is FIRRTL ground: Hardware construction languages, compiler frameworks, and transformations," Adam Izraelevitz et al., ICCAD 2017](https://ieeexplore.ieee.org/abstract/document/8203780)

## FIRRTL "Lowers" Abstractions Gradually

* FIRRTL library uses _passes_ to slowly change IR to desired result
  * Breaks challenging problem of making a compiler into making many manageable passes or transformations
    * With single purpose, each pass is easier to develop & test
  * Another best practice from compiler community
* FIRRTL formalizes abstraction complexity with multiple _levels_
  * Lower levels are a subset of nodes and values of higher levels
  * _Lowering_ - _High-level_ abstractions (e.g. Chisel) -> _Low-level_ concrete things (e.g. Verilog)
* _High FIRRTL_ - all node types available & many bitwidths are unspecified
  * Like what is emitted by Chisel frontend
* _Low FIRRTL_ - like Verilog - subset of node types available & all bitwidths are specified
  * Easy for Verilog Backend to emit as Verilog
* In addition to lowering, transformations can: analyze, optimize, instrument, specialize

## Example FIRRTL Operations

#### Lowering Passes
* _Infer Widths_ - iteratively determines what final widths of signals should be
* _Pad widths_ - ensure all operators are given operands of the same width
* _Expand Whens_ - replaces when statements with appropriate connections and muxes
* Many safety checks are performed by FIRRTL library instead of Chisel frontend

#### Optimization Transformations
* _Constant propagation_ - replace references to literals with literals and simplify logic
* _Dead code elimination (DCE)_ - remove disconnected statements and modules
* _Common subexpression elimination (CSE)_ - deduplicate repeated expressions

## FireSim & Golden Gate - Large Example of Custom Tools

* [FireSim](https://fires.im) - automates process of simulating Chisel designs on cloud FPGAs
  * Virtualizes simulation time relative to the passage of time on the physical FPGA
  * Under-the-hood, instruments/modifies design for simulation and even tracking
* [Golden Gate](https://people.eecs.berkeley.edu/~biancolin/papers/goldengate-iccad19.pdf)  improves simulation resource efficiency by sharing/multiplexing
* Both tool flows leverage FIRRTL library and development focuses on unique optimizations/transformations and were able to reuse much of the rest

## ESSENT - Another FIRRTL User

* [ESSENT](https://github.com/ucsc-vama/essent) is a FIRRTL simulator, that is the fastest cycle-accurate software RTL simulator
  * Is also one of main research thrusts of the course instructor's group :)
* By leveraging `firrtl` library, we are able to focus on the novel innovations for our simulation approach
  * ESSENT code is only around ~5K lines of Scala (not counting firrtl)
  * Current leading open-source Verilog simulator (Verilator is >100K lines of C++)
    * ESSENT outperforms it by 2x by using more sophisticated optimizations
    * Verilator does support some Verilog constructs not expressable in FIRRTL

## Summary

### Hardware IRs enable designers to make tool-like transformations

### There is a growing landscape of hardware IRs
  * [FIRRTL](https://github.com/chipsalliance/firrtl) - today's lecture, supports Chisel
  * [RTLIL](http://www.clifford.at/yosys/) - from Yosys
  * [CoreIR](https://github.com/rdaly525/coreir) - from Stanford
  * [LLHD](http://www.llhd.io) - from ETH
  * [CIRCT](https://github.com/llvm/circt) - from LLVM